In [70]:
import numpy as np
import pickle

with open('Adv_sample.pkl','rb') as f:
    X_adv=pickle.load(f)

In [71]:
X_adv

array([[0.        , 0.        , 0.        , ..., 0.        , 0.0035203 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98855877, 0.15069449, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11555127, 0.11555127, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [72]:
X_adv.shape

(23516, 36)

In [73]:
#original data
with open('X_train_scaled.pkl','rb') as X_tr:
    X_train_scaled=pickle.load(X_tr)
with open('X_test_scaled.pkl','rb') as X_te:
    X_test_scaled=pickle.load(X_te)
with open('y_train.pkl','rb') as y_tr:
    Y_train=pickle.load(y_tr)
with open('y_test.pkl','rb') as y_te:
    Y_test=pickle.load(y_te)
    

In [74]:
print(X_train_scaled.shape,Y_train.shape,X_test_scaled.shape, Y_test.shape)

(125000, 36) (125000,) (23516, 36) (23516,)


In [75]:
Y_train

1         normal
2         normal
3         attack
4         normal
5         normal
6         attack
7         attack
8         attack
9         attack
10        attack
11        attack
12        attack
13        normal
14        attack
15        attack
16        attack
17        normal
18        attack
19        normal
20        normal
21        attack
22        attack
23        normal
24        normal
25        attack
26        normal
27        attack
28        normal
29        normal
30        normal
           ...  
124971    normal
124972    normal
124973    normal
124974    attack
124975    attack
124976    normal
124977    normal
124978    attack
124979    normal
124980    attack
124981    normal
124982    attack
124983    attack
124984    normal
124985    normal
124986    attack
124987    attack
124988    normal
124989    attack
124990    attack
124991    attack
124992    attack
124993    normal
124994    attack
124995    normal
124996    normal
124997    attack
124998    norm

In [76]:
def convert_bool(x):
    if(x == "attack"):
        return 1
    else:
        return 0
    

In [77]:
Y_train = np.array(Y_train.apply(convert_bool))
Y_test=np.array(Y_test.apply(convert_bool))

In [78]:
Y_train.shape

(125000,)

In [79]:
Y_test

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

Divide X_adv in two parts X_train and X_test             Divide X_test_scaled in two parts y_train and y_test

In [80]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_adv, X_test_scaled, test_size=0.2, random_state=4)

In [81]:
X_train

array([[0.12031003, 0.11246372, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28911734, 0.7131561 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.17635958, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07755633, 0.9899838 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00340742,
        0.        ]])

In [82]:
y_train

array([[0.12031003, 0.11246372, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28911734, 0.7131561 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.17635958, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07755634, 0.98998381, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00340742,
        0.        ]])

In [83]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(18812, 36) (18812, 36) (4704, 36) (4704, 36)


# Model for de-noising

In [84]:
from keras.layers import Input, Dense 
from keras.models import Model, load_model

In [85]:
#encoding_dim = 10
input_dim = X_adv.shape[-1]

In [86]:
input_dim

36

In [131]:
#model for encoding adversarial input
def auto_encoder_model(input_dim) :
    
    input_layer = Input(shape = (input_dim,))
    
    encoder = Dense(18, activation = 'relu')(input_layer)
    #encoder = Dense(12, activation="relu")(encoder)
    #encoder = Dense(9, activation="relu")(encoder)
    encoder = Dense(3, activation="relu")(encoder)
    
    #decoder = Dense(6 ,activation='relu')(encoder)
    #decoder = Dense(9, activation='relu')(encoder)
    decoder = Dense(18, activation='relu')(encoder)
    decoder = Dense(input_dim, activation='relu')(decoder)
    
    autoencoder = Model(inputs=input_layer,outputs= decoder)
    
    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    autoencoder.summary()
    return autoencoder


In [132]:
#model for encoder
denoiser = auto_encoder_model(input_dim)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 36)                0         
_________________________________________________________________
dense_75 (Dense)             (None, 18)                666       
_________________________________________________________________
dense_76 (Dense)             (None, 3)                 57        
_________________________________________________________________
dense_77 (Dense)             (None, 18)                72        
_________________________________________________________________
dense_78 (Dense)             (None, 36)                684       
Total params: 1,479
Trainable params: 1,479
Non-trainable params: 0
_________________________________________________________________


In [140]:
denoiser.fit(X_train, y_train, epochs=100, batch_size=50, validation_data = (X_test, y_test), )

Train on 18812 samples, validate on 4704 samples
Epoch 1/100
18812/18812 [==============================] - 1s 74us/step - loss: 4.4568e-04 - acc: 0.9296 - val_loss: 5.1532e-04 - val_acc: 0.9154
Epoch 2/100
18812/18812 [==============================] - 1s 72us/step - loss: 4.4383e-04 - acc: 0.9303 - val_loss: 5.3015e-04 - val_acc: 0.9624
Epoch 3/100
18812/18812 [==============================] - 1s 75us/step - loss: 4.3639e-04 - acc: 0.9306 - val_loss: 5.0204e-04 - val_acc: 0.9339
Epoch 4/100
18812/18812 [==============================] - 2s 93us/step - loss: 4.3467e-04 - acc: 0.9280 - val_loss: 5.0122e-04 - val_acc: 0.9247
Epoch 5/100
18812/18812 [==============================] - 2s 81us/step - loss: 4.3017e-04 - acc: 0.9287 - val_loss: 4.9923e-04 - val_acc: 0.9447
Epoch 6/100
18812/18812 [==============================] - 1s 79us/step - loss: 4.2973e-04 - acc: 0.9289 - val_loss: 4.9009e-04 - val_acc: 0.9449
Epoch 7/100
18812/18812 [==============================] - 1s 61us/step - l

18812/18812 [==============================] - 2s 82us/step - loss: 3.6060e-04 - acc: 0.9261 - val_loss: 4.5011e-04 - val_acc: 0.9447
Epoch 57/100
18812/18812 [==============================] - 2s 84us/step - loss: 3.6149e-04 - acc: 0.9273 - val_loss: 4.2727e-04 - val_acc: 0.9116
Epoch 58/100
18812/18812 [==============================] - 2s 90us/step - loss: 3.5900e-04 - acc: 0.9262 - val_loss: 4.2669e-04 - val_acc: 0.9384
Epoch 59/100
18812/18812 [==============================] - 1s 79us/step - loss: 3.5767e-04 - acc: 0.9274 - val_loss: 4.3247e-04 - val_acc: 0.9279
Epoch 60/100
18812/18812 [==============================] - 1s 79us/step - loss: 3.5815e-04 - acc: 0.9272 - val_loss: 4.2895e-04 - val_acc: 0.9403
Epoch 61/100
18812/18812 [==============================] - 2s 85us/step - loss: 3.5730e-04 - acc: 0.9237 - val_loss: 4.2798e-04 - val_acc: 0.9233
Epoch 62/100
18812/18812 [==============================] - 2s 80us/step - loss: 3.5654e-04 - acc: 0.9281 - val_loss: 4.2428e-04 - 

In [141]:
#predicting original test_scaled
X_adv_denoised=denoiser.predict(X_adv)

In [142]:
X_adv

array([[0.        , 0.        , 0.        , ..., 0.        , 0.0035203 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98855877, 0.15069449, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11555127, 0.11555127, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [143]:
X_test_scaled

array([[0.        , 0.        , 0.        , ..., 0.        , 0.0035203 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98855875, 0.15069449, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11555127, 0.11555127, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [144]:
X_adv_denoised

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01216458, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9982251 , 0.15714976, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13139431, 0.12650296, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [145]:
X_adv_denoised.shape

(23516, 36)

In [146]:
import pickle
with open('X_adv_denoiser.pkl','wb') as f:
    pickle.dump(X_adv_denoised,f)